In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

MLP(Multilayer Perceptron) 多重感知机 ，也可以称为『多层全连接网络』  
这里使用Minst手写体数据集进行实验

# 数据获取及预处理
使用tf.keras,datasets

在tensorflow中，图像数据集的一种典型表示为\[图像数目， 长， 宽， 色彩通道数\] 的四维张量。  
mnist 中图像的大小为 28*28， 灰度图像，训练集60000张， 测试集10000张  
因此train_data为 \[60000, 28, 28\]这种形式  
需要使用expand_dims给最后添加一维色彩通道  

In [4]:
class MNISTLoader():
    def __init__(self):
        mnist = keras.datasets.mnist
        # MNIST 中的图像默认为 unit8(0-255)，需要归一化到0-1，并增加一维颜色通道
        (self.train_data, self.train_label),(self.test_data, self.test_label) = mnist.load_data()
        # axis=-1表示最后一维，维数可以简单理解为下标
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1)

        self.train_label = self.train_label.astype(np.float32)
        self.test_label = self.test_label.astype(np.float32)

        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从数据集中随机抽取batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        # TODO 解释train_data[index, :]的含义，需要看数据格式,
        return self.train_data[index, :], self.train_label[index]

np.random.randint(low, high, size)   
返回size个在low ~ high之间的数  
满足均匀分布，原则上每个batch被取到的次数相同，直接对数据进行shuffle也可，不过比较消耗内存

## 模型构建MLP
这里使用tf.keras.Model, 需要重写 __init__（）函数和 call()函数

In [14]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # flatten层将输入的矩阵压平: 将除第一维batch_size以外的平度展平
        self.flatten = layers.Flatten()
        # 两层全连接层
        self.dense1 = layers.Dense(100, activation=tf.nn.relu)
        # 第二层的单元个数为分类的个数0-9十个数字
        self.dense2 = layers.Dense(10)

    def call(self, inputs):
        # 第一层处理
        x = self.flatten(inputs)
        # 经过两层的全连接层
        x = self.dense1(x)
        x = self.dense2(x)
        # 输出值经过softmax转化为预测的概率值
        outputs = tf.nn.softmax(x)
        return outputs

## 模型训练
需要先定义一些模型的超参数，例如batch_size, epoches, learning_rate等  
其实全链接层的单元数也可以作为超参传入 

In [41]:
num_epoches = 5
batch_size = 50
learning_rate = 0.001

# 实例化模型和数据读取类
data_loader = MNISTLoader()
mlp_model = MLP()

# 实例化优化器，使用SGD
optimizer = optimizers.Adam(learning_rate=learning_rate)

# mlp_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=tf.keras.metrics.SparseCategoricalAccuracy)

In [38]:
print("train_data: %s" % data_loader.num_train_data)
print("test_data: %s" % data_loader.num_test_data)

train_data: 60000
test_data: 10000


## 迭代步骤
- 从DataLoader中随机取batch
- 将数据送入模型，计算模型的预测值
- 将模型的预测值与真实值进行比较，计算Loss
- 计算损失函数关于模型变量的导数
- 将求出的导数出入优化器，使用apply_gradients更新参数以最小化损失函数

In [43]:
# 总计的batch数量,使用tf.keras.Dataset.from_tensor_slices时生成pipline,直接在pipline中指定batch_size即可
num_batches = int(data_loader.num_train_data//batch_size * num_epoches)
# 循环喂数据
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    # 模型训练
    with tf.GradientTape() as tape:
        # 默认调用call函数
        y_pred = mlp_model(X)
        # 使用分类交叉熵函数作为损失函数(适合稀疏特征)
        loss = losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        # 每个分类预测的平均值
        loss = tf.reduce_mean(loss)
        if batch_index % 500 == 0:
            print("batch %d \t loss: %f" % (batch_index, loss))
    # 更新梯度
    grads = tape.gradient(loss, mlp_model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, mlp_model.variables))
    


batch 0 	 loss: 2.455234
batch 500 	 loss: 0.168660
batch 1000 	 loss: 0.087254
batch 1500 	 loss: 0.071638
batch 2000 	 loss: 0.108226
batch 2500 	 loss: 0.153149
batch 3000 	 loss: 0.159102
batch 3500 	 loss: 0.238657
batch 4000 	 loss: 0.024842
batch 4500 	 loss: 0.115302
batch 5000 	 loss: 0.053679
batch 5500 	 loss: 0.004715


tf.keras中， 有两个交叉熵有关的损失函数  
- categorical_crossentropy
- sparse_categorical_croosentrop

其中sparse的含义是，真实的标签值可以直接传入int类型的标签类别。
等价于
```python
categorical_crossentropy(y_true=tf.one_hot(y,depth=tf.shape(y_pred)[-1]), y_pred=y_pred)
```

## 模型的评估
使用tf.keras.metrics来进行评估模型的效果  
这里使用SparseCagtegoricalAccurary评估器来评估在测试集上的性能
该评估器能够输出预测正确的样本数占总样本的比例  
通过update_state()方法向评估器传入 y_pred 和 y_true

In [44]:
def acc():
    # 定义检测指标对象
    sparse_acc = tf.keras.metrics.SparseCategoricalAccuracy()
    batch_num= int(data_loader.num_test_data//batch_size)

    for batch_index in range(batch_num):
        start_index, end_index = batch_index * batch_size, (batch_index+1)*batch_size
        y_pred = mlp_model.predict(data_loader.test_data[start_index: end_index])
        sparse_acc.update_state(data_loader.test_label[start_index:end_index], y_pred)
    print("test_acc: %f" % sparse_acc.result())
acc()

test_acc: 0.974100


# 使用keras 实现卷积神经网络
相对MLP，卷积神经网络加入了一些卷积层和池化层。  
这里的网络结构并不唯一，可以增加、删除调整CNN的网络结构和参数   
以达到更好的性能